In [14]:
import numpy as np
import random
import pandas as pd

In [132]:
# def set_reservation_prices(box_df, search_cost):
#     """
#     Generates a distribution of reservation prices using Weitzman's optimal search framework.

#     Args:
#     distributions (list): A list of tuples (mean, std_dev) representing the parameters of normal distributions for each alternative.
#     search_cost (float): The cost of searching for a new alternative.

#     Returns:
#     list: A list of reservation prices for each alternative.
#     """

#     box_means = box_df['mean']
#     box_stds = box_df['std_dev']

#     distributions = list(zip(box_means, box_stds))
#     print(len(distributions))

#     reservation_prices = []

#     for mean, std_dev in distributions:
#         # Calculate the Gittins index for each alternative
#         gittins_index = mean - (search_cost * (std_dev ** 2))

#         # Calculate the reservation price for each alternative
#         reservation_price = gittins_index * (1 - np.exp(-2 * search_cost / (std_dev ** 2)))
#         reservation_prices.append(reservation_price)

#     box_df['reservation_price'] = reservation_prices



In [139]:
from scipy.optimize import minimize_scalar

def utility(x_S, y):
    return y + sum(x_S)

def expected_utility(x_S, mu_i, sigma_i):
    samples = np.random.normal(mu_i, sigma_i, 10000)
    utilities = [utility(x_S, sample) for sample in samples]
    return np.mean(utilities)

def find_reservation_price(c_i, mu_i, sigma_i, x_S):
    def to_minimize(y):
        return abs(utility(x_S, y) + c_i - expected_utility(x_S, mu_i, sigma_i))
    result = minimize_scalar(to_minimize)
    if result.success:
        return result.x
    else:
        return None

def set_reservation_prices(box_df, search_cost):
    box_means = box_df['mean']
    box_stds = box_df['std_dev']
    x_S = []
    reservation_prices = []
    for mean, std_dev in zip(box_means, box_stds):
        reservation_price = find_reservation_price(search_cost, mean, std_dev, x_S)
        reservation_prices.append(reservation_price)
        x_S.append(mean)
    box_df['reservation_price'] = reservation_prices

In [140]:
import numpy as np

# Generate 10 mean values that increase exponentially from 20 to 110
mean_values = np.logspace(np.log10(20), np.log10(110), num=10)

# Generate 10 standard deviations that range from 5 to 50
std_devs = np.linspace(1, 3, num=10)

# Combine the mean values and standard deviations into a list of boxes
boxes = list(zip(mean_values, std_devs))


box_df = pd.DataFrame(boxes, columns=['mean', 'std_dev'])
box_df['cost'] = 10

rewards = []
reward_sum = 0
cost_sum = 0

set_reservation_prices(box_df, 10)

#print(box_df)


for index, row in box_df.iterrows(): 
    if row['reservation_price'] < row['mean']:
        reward = np.random.normal(row['mean'], row['std_dev'])
        rewards.append(reward)
        reward_sum += reward
        cost_sum += row['cost']
        box_df.drop(index, inplace=True)
    
        set_reservation_prices(box_df, 10)
    print(box_df)

print(f"Total reward: {reward_sum}")

print(f"Total cost: {cost_sum}")

print(f"Net reward: {reward_sum - cost_sum}")

print(box_df)

         mean   std_dev  cost  reservation_price
1   24.170883  1.222222    10          14.179883
2   29.211579  1.444444    10          19.207031
3   35.303483  1.666667    10          25.322370
4   42.665818  1.888889    10          32.657033
5   51.563525  2.111111    10          41.580446
6   62.316797  2.333333    10          52.305993
7   75.312600  2.555556    10          65.326551
8   91.018603  2.777778    10          81.005164
9  110.000000  3.000000    10          99.986888
         mean   std_dev  cost  reservation_price
2   29.211579  1.444444    10          19.201527
3   35.303483  1.666667    10          25.288964
4   42.665818  1.888889    10          32.618016
5   51.563525  2.111111    10          41.583575
6   62.316797  2.333333    10          52.338358
7   75.312600  2.555556    10          65.334187
8   91.018603  2.777778    10          81.023939
9  110.000000  3.000000    10          99.941710
         mean   std_dev  cost  reservation_price
3   35.303483  1.666

In [123]:
# Initialize list of already opened boxes
x_S = []

# Initialize total cost
total_cost = 0

# Calculate initial reservation prices
box_df['reservation_price'] = box_df.apply(lambda row: find_reservation_price(row['cost'], row['mean'], row['std_dev'], x_S), axis=1)

while not box_df.empty:
    # Find the box with the lowest reservation price
    min_price_row = box_df.loc[box_df['reservation_price'].idxmin()]

    # If the reservation price is less than or equal to the mean value in the box, open the box
    if min_price_row['reservation_price'] <= min_price_row['mean']:
        # Add value in box to list of already opened boxes
        x_S.append(min_price_row['mean'])

        # Add cost of opening box to total cost
        total_cost += min_price_row['cost']

        # Remove box from dataframe
        box_df.drop(min_price_row.name, inplace=True)

        # Update reservation prices for remaining boxes
        box_df['reservation_price'] = box_df.apply(lambda row: find_reservation_price(row['cost'], row['mean'], row['std_dev'], x_S), axis=1)
    else:
        break

    print(box_df)

         mean  std_dev  cost  reservation_price
0   20.000000      5.0    10          10.034664
1   24.170883     10.0    10          13.928155
2   29.211579     15.0    10          19.225420
3   35.303483     20.0    10          25.112842
4   42.665818     25.0    10          32.628158
5   51.563525     30.0    10          41.517914
7   75.312600     40.0    10          -0.005025
8   91.018603     45.0    10          80.788548
9  110.000000     50.0    10          98.723192
         mean  std_dev  cost  reservation_price
0   20.000000      5.0    10           9.990522
1   24.170883     10.0    10          14.254475
2   29.211579     15.0    10          19.238950
3   35.303483     20.0    10          25.166697
4   42.665818     25.0    10          32.317909
5   51.563525     30.0    10          41.245947
8   91.018603     45.0    10           7.393247
9  110.000000     50.0    10         102.476175
         mean  std_dev  cost  reservation_price
0   20.000000      5.0    10           9

ValueError: Columns must be same length as key

In [97]:
box_df

,mean,std_dev,reservation_price


In [89]:
rewards

[20.0, 30.0, 40.0, 50.0, 60.0, 70.0, 80.0, 90.0, 100.0, 110.0]

In [88]:
box_df

,mean,std_dev,reservation_price
0,20,5,19.000000
1,30,10,29.000000
2,40,15,39.000000
3,50,20,49.000001
4,60,25,59.000000
5,70,30,69.000000
6,80,35,79.000000
7,90,40,89.000000
8,100,45,99.000000
9,110,50,108.999999


In [ ]:
def open_Box(box): 

In [46]:
def open_Box(box):
    # Simulate the value of the box

   # box['opened'] = True

    return random.gauss(box['mean'], box['std_dev'])

def remove_box(box_df, index):
    # Remove the box from the dataframe
    box_df.drop(index, inplace=True)


def search_boxes(box_df, search_cost):

    """
    Searches for the best alternative given a list of reservation prices.

    Args:
    boxes (list): A list of tuples (mean, std_dev) representing the parameters of normal distributions for each alternative.
    reservation_prices (list): A list of reservation prices for each alternative.

    Returns:
    int: The index of the best alternative.
    """

    best_value = 0
    best_index = -1

    for i, box in box_df.iterrows():
        
        print(box_df)
        # Open the box and check its value
        value = open_Box(box)

        print(f'Value of box {i}: {value}')
        print(f'Reservation price of box {i}: {box["reservation_price"]}')

        # Check if the value is greater than the reservation price
        if value > box['reservation_price'] and value > best_value:
            best_value = value
            best_index = i

        remove_box(box_df, i)
        set_reservation_prices(box_df, search_cost)

    return best_index

# Example usage 


set_reservation_prices(box_df, search_cost)

In [47]:
search_boxes(box_df, search_cost)

   mean  std_dev  reservation_price
0    10        2           2.360816
1    15        3           1.195576
2    20        4           0.470012
Value of box 0: 12.522626545028846
Reservation price of box 0: 2.3608160417241995
   mean  std_dev  reservation_price
1    15        3           1.195576
2    20        4           0.470012
Value of box 1: 20.842696889111213
Reservation price of box 1: 1.1955755824991516
   mean  std_dev  reservation_price
2    20        4           0.470012
Value of box 2: 16.23675507601765
Reservation price of box 2: 0.4700123896616182


1

In [8]:
def gen_Box_Val(p, full_val):
    val = random.choice([0, full_val, p])
    return val    

In [17]:
box = gen_Box_Val(0.1, 100)

In [19]:
def genVideoBoxes():

    import numpy as np
    import pandas as pd

    np.random.seed(42)  # For reproducibility

    # Define number of boxes
    n_boxes = 100

    # Define mean and standard deviation for prize and cost distributions
    prize_mean = 100
    prize_std = 20
    cost_mean = 10
    cost_std = 2

    # Generate prizes and costs
    prizes = np.random.normal(prize_mean, prize_std, n_boxes)
    costs = np.random.normal(cost_mean, cost_std, n_boxes)

    # Make sure that costs and prizes are not negative
    costs = np.abs(costs)
    prizes = np.abs(prizes)

    # Create a DataFrame for better data management
    df = pd.DataFrame({
        'prize': prizes,
        'cost': costs
    })

    return df

def solveVideoBoxes(vdf):
    # Calculate indices
    vdf['index'] = vdf['prize'] - vdf['cost']

    total_prize = 0
    total_cost = 0

    while len(vdf) > 0:
        # Find the box with the maximum index
        max_index_box = vdf['index'].idxmax()

        # If the index is positive, open the box
        if vdf.loc[max_index_box, 'index'] > 0:
            total_prize += vdf.loc[max_index_box, 'prize']
            total_cost += vdf.loc[max_index_box, 'cost']
            vdf.drop(max_index_box, inplace=True)
        else:
            break

    print("Total prize:", total_prize)
    print("Total cost:", total_cost)
    print("Net gain:", total_prize - total_cost)


In [26]:
vdf = genVideoBoxes()
print(len(vdf))

100


In [27]:
solveVideoBoxes(vdf)
print(len(vdf))

Total prize: 9792.306965211807
Total cost: 1004.460917409985
Net gain: 8787.846047801822
0


In [21]:
vdf

,prize,cost
0,109.934283,7.169259
1,97.234714,9.158709
2,112.953771,9.314571
3,130.460597,8.395445
4,95.316933,9.677429
...,...,...
95,70.729701,10.770635
96,105.922406,8.232285
97,105.221105,10.307450
98,100.102269,10.116417


In [22]:
zipped = list(zip(vdf['prize'], vdf['cost']))

In [23]:
zipped[0][0]

109.93428306022466

In [24]:
len(zipped)

100

In [35]:
for i in range(1, (len(zipped)+1)):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [36]:
import itertools

set1 = range(11)  # integers from 0 through 10
set2 = range(11)  # integers from 0 through 10

# Get all combinations of integers from set1 and set2
combinations = list(itertools.product(set1, set2))

print(combinations)


[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6), (6, 7), (6, 8), (6, 9), (6, 10), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 7), (7, 8), (7, 9), (7, 10), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 8), (8, 9), (8, 10), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9), (9, 10), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9), (10, 10)]


In [48]:
for i in range(len(zipped)):
   box = random.sample(zipped, 1)[0]
   print(box)
   zipped.remove(box)

(89.11234550949635, 11.17371418760054)
(61.73439510684404, 14.926484224970572)
(101.83521553071004, 9.470686333524087)
(112.23352577681736, 10.464507394322007)
(116.25051644788397, 11.926752258488644)
(106.87236579136923, 11.043883131233795)
(104.1772719000951, 8.433493415327526)
(60.80659752240449, 9.35587696758865)
(87.09760490789752, 8.492527671285021)
(77.87330051987944, 12.317191158014808)
(81.83951848957578, 12.285645629030041)
(107.23272050095268, 9.509223767994259)
(101.74094136476343, 10.026003783755813)
(106.4816793878959, 10.500985700691754)
(71.75392597329417, 11.503866065373549)
(89.63459563452705, 10.96494483048637)
(60.248621707982146, 15.440338333179238)
(105.92240554129152, 8.232285127597734)
(129.55788089483033, 7.858215003877776)
(99.28347921780097, 9.845796581171792)
(99.73005550524132, 7.87539257254779)
(90.78722458080425, 7.526098578243836)
(76.98012845155395, 8.018927349738624)
(89.40479592465923, 8.30641256386319)
(94.01985299068265, 12.907068154314633)
(116.450

In [45]:
box_values

(71.75392597329417, 11.503866065373549)

In [57]:
import numpy as np

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def choose_based_on_acuity(choice_set, acuity):
    # Normalize the choice set
    normalized_choices = (choice_set - np.min(choice_set)) / (np.max(choice_set) - np.min(choice_set))

    # Multiply by the acuity
    acuity_adjusted_choices = normalized_choices * (acuity / 100)
    print(acuity_adjusted_choices)

    # Calculate probabilities using softmax
    probabilities = softmax(acuity_adjusted_choices)

    # Choose an element from the choice set based on the probabilities
    choice = np.random.choice(choice_set, p=probabilities)

    return choice, probabilities


In [59]:
choice = choose_based_on_acuity([0, 25, 75, 100], 0)

[0. 0. 0. 0.]
